In [1]:
import urllib
from urllib import request
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options  
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os.path
import os
import shutil
import glob
from pathlib import Path


In [2]:
# company_number = '7526411'
# company_number = '7529401'

In [3]:
# save_path = 'S:\\Actuarial\\Python\\WebScraping\\UK_Company\\'
# pdf_save_path = 'S:\\Actuarial\\Python\\WebScraping\\UK_Company'

# save_path = 'C:\\Users\\Colin\\WebScraping\\UK_Company\\'
# pdf_save_path = 'C:\\Users\\Colin\\WebScraping\\UK_Company'


In [4]:
class get_company_info():

    
    def __init__ (self):
        self.office_address = x
     
    def company_name(html):
        soup = BeautifulSoup(html,"lxml")
        tag = 'p'
        tag_name = "company-name"
        result = soup.find(tag,id = {tag_name})        
        return (result.text.strip())
    
    def company_status(html):
        soup = BeautifulSoup(html,"lxml")
        tag = 'dd'
        tag_name = "company-status"
        result = soup.find(tag,id = {tag_name})        
        return (result.text.strip())
    
    def office_address(html):
        soup = BeautifulSoup(html,"lxml")
        tag = 'dd'
        tag_name = "text data"
        result = soup.find(tag,class_ = {tag_name})        
        return (result.text.strip())
    
    def company_type(html):
        soup = BeautifulSoup(html,"lxml")
        tag = 'dd'
        tag_name = "company-type"
        result = soup.find(tag,id = {tag_name})        
        return (result.text.strip())

    def last_accounts_date(html):
        soup = BeautifulSoup(html,"lxml")
        tag = 'strong'
        #tag_name = "heading_medium"
        result = soup.find_all(tag) 
        #Bit messy
        return (result[5].text.strip())  
    
    def last_statement_date(html):
        soup = BeautifulSoup(html,"lxml")
        tag = 'strong'
        tag_name = "text data"
        try:
            result = soup.find_all(tag) 
        #Bit messy
            return (result[9].text.strip())  
        except IndexError:  
            return "Last statement date not found"
            
    def nature_of_business(html):
        soup = BeautifulSoup(html,"lxml")
        tag = 'span'
        tag_name = "sic0"
        result = soup.find(tag,id = {tag_name})        
        return (result.text.strip())


In [5]:
def close_ad():
    #For when popup ad appears 
    try:
        page_wait = WebDriverWait(driver,2)
        ad_button_xpath = "//a[@class = 'acsCloseButton acsAbandonButton ']"
        ad_button = page_wait.until(EC.visibility_of_element_located((By.XPATH, "//a[@class = 'acsCloseButton acsAbandonButton ']")))       
        print("ad button found")
        ad_button.click()
    except TimeoutException:
        #print("")
        None
        #More error catching needed?

In [70]:
def go_to_website(company_number,output_path,hidden_flag):
    
    chrome_options = Options()  
    
    if hidden_flag == True:
        chrome_options.add_argument("--headless")
    
    chrome_options.add_experimental_option("prefs", {
      "download.default_directory": output_path,
      "download.prompt_for_download": False,
      "download.directory_upgrade": True,
      "safebrowsing.enabled": True ,
      "add_experimental_option": True
    })
    
    chromedriver_path = 'C:/Program Files (x86)/ChromeDriver/chromedriver_win32/chromedriver.exe'

    driver = webdriver.Chrome(executable_path = chromedriver_path, chrome_options = chrome_options)

    driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
    params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': output_path}}
    command_result = driver.execute("send_command", params)

    login_wait = WebDriverWait(driver, 10)
    
    webpage = 'https://beta.companieshouse.gov.uk//search//companies'
    driver.get(webpage)
    html = driver.page_source
    company_text_area_id = 'site-search-text'
    search_button_id = 'search-submit'
    login_wait = WebDriverWait(driver, 10)
    
    
    
    #close_ad()
    try:  
        elem = login_wait.until(EC.visibility_of_element_located((By.ID, company_text_area_id)))
        print("Page is ready!")
        company_text_area = driver.find_element_by_id(company_text_area_id)
        driver.execute_script("arguments[0].scrollIntoView();", company_text_area)
        company_text_area.send_keys(company_number)

    except TimeoutException:
        print("Timeout Exception")  

    search_button = driver.find_element_by_id(search_button_id)

    search_button.click()

    company_link_href = '/company/' + str(company_number).zfill(8) 
    company_link_xpath = "//a[@href =  '" +  company_link_href + "']"

    #close_ad()

    try:  

        company_link_xpath = "//a[@href =  '" +  company_link_href + "']"
        elem = login_wait.until(EC.visibility_of_element_located((By.XPATH, company_link_xpath)))
    #     print("Page is ready!")    
        company_link = driver.find_element_by_xpath(company_link_xpath)  
        company_link.click();    

    except TimeoutException:
        print("Loading took too much time company link")   
        
    
    
    html = driver.page_source
    #soup = BeautifulSoup(html,"lxml")
    #item_id = "company-status"
    #result = soup.find('dd',id = {item_id})
    #print(result.text.strip())
    
    company_name = get_company_info.company_name(html)
    company_status = get_company_info.company_status(html)
    nature_of_business = get_company_info.nature_of_business(html)
    office_address = get_company_info.office_address(html) 
    company_type = get_company_info.company_type(html)
    last_accounts_date = get_company_info.last_accounts_date(html)
    last_statement_date = get_company_info.last_statement_date(html)
    
    company_info_list = []
    company_info_list.append(company_name)
    company_info_list.append(company_status)
    company_info_list.append(nature_of_business)
    company_info_list.append(office_address)
    company_info_list.append(company_type)
    company_info_list.append(last_accounts_date)
    company_info_list.append(last_statement_date)
    
    
    last_accounts_year = last_accounts_date[-4:]

    filing_tab_id = 'filing-history-tab'
    filing_tab_button = driver.find_element_by_id(filing_tab_id)
    filing_tab_button.click()

    accounts_checkbox_id = 'filter-category-accounts'
    if not(driver.find_element_by_id(accounts_checkbox_id).is_selected()):   
        driver.find_element_by_id(accounts_checkbox_id).click()

    WebDriverWait(driver, 2)
    
    
    pdf =  driver.find_elements_by_class_name("download")[0]
    #script = "arguments[0].setAttribute('download','" + pdf_save_path + "');"
    script = "arguments[0].setAttribute('download','" + output_path + "');"
    
    driver.execute_script(script, pdf);
    pdf.click(); 
    
    #print("Saved to " + pdf_save_path)
    print("Saved to " + output_path)
    time.sleep(2) 
    change_name(output_path,company_number)
    
    
    return company_name
    

In [59]:
# p = Path(os.getcwd()).parents[0]
# filepath = str(p / "PDF")
# company_number = '7526411'


In [72]:
# go_to_website(company_number,pdf_path,True)

Page is ready!
Saved to C:\Users\Colin\Documents\Python\WebScraping\UK_Company_Combined\PDF
C:\Users\Colin\Documents\Python\WebScraping\UK_Company_Combined\PDF
C:\Users\Colin\Documents\Python\WebScraping\UK_Company_Combined\PDF\application-pdf.pdf
C:\Users\Colin\Documents\Python\WebScraping\UK_Company_Combined\PDF\7526411.pdf


'FLYING B PRODUCTIONS LTD'

In [61]:
# change_name(pdf_path,company_number)

C:\Users\Colin\Documents\Python\WebScraping\UK_Company_Combined\PDF
C:\Users\Colin\Documents\Python\WebScraping\UK_Company_Combined\PDF\application-pdf.pdf
C:\Users\Colin\Documents\Python\WebScraping\UK_Company_Combined\PDF\7526411.pdf


In [64]:
def change_name(filepath,new_name):
    #Change most recent file to number
    filename = max(glob.iglob(filepath+'\\*.[Pp][Dd][Ff]'), key=os.path.getctime)
    print(filepath)
    print(filename)
    shutil.move(filename,filepath + '\\' + new_name+'.pdf')
    print(filepath + '\\' + new_name+'.pdf')
    